In [38]:
from itertools import combinations
from collections import Counter
import random

In [100]:
## Normalises a pool dictionary
def Pnorm(pool):
    total = sum([1 / s[0] for s in pool.values()])
    for team in pool.keys():
        pool[team][0] = (1 / pool[team][0]) / total
    return pool

## Normalises a full tournament dictionary
def Tnorm(pool):
    total = sum([1 / s[1] for s in pool.values()])
    for team in pool.keys():
        pool[team][1] = (total - (1 / pool[team][1])) / total
    return pool

We put the odds for each team based on their pooling. [a,b,c] = ['Odds within pool', 'Odds overall', score]

In [101]:
Teams = ['Ireland', 'Scotland', 'Japan', 'Russia', 'Samoa',
         'New Zealand', 'South Africa', 'Italy', 'Namibia', 'Canada',
         'England', 'France', 'Argentina', 'Tonga', 'USA',
         'Wales', 'Australia', 'Fiji', 'Georgia', 'Uruguay']
PoolA = Pnorm({'Ireland':[2246,4.7,0], 'Scotland':[2129,24,0], 'Japan':[2050,60,0], 'Russia':[1789,200,0], 'Samoa':[1965,200,0]})
PoolB = Pnorm({'New Zealand':[2323,1.76,0], 'South Africa':[2201,2.88,0], 'Italy':[1889,100,0], 'Namibia':[1838,200,0], 'Canada':[1839,100,0]})
PoolC = Pnorm({'England':[1.38,3.5,0], 'France':[5.6,12.5,0], 'Argentina':[7.2,20,0], 'Tonga':[500,100,0], 'USA':[500,100,0]})
PoolD = Pnorm({'Wales':[1.86,4.6,0], 'Australia':[2.12,6.8,0], 'Fiji':[40,60,0], 'Georgia':[500,100,0], 'Uruguay':[500,200,0]})
quarter_play = [(4,7), (1,2), (6,5), (0,3)]
semi_play = [(0,1),(2,3)]

In [57]:
def who_won(teamA, teamB, probA, probB, loser = False):
    if loser:
        return (teamA, teamB) if random.random() < 1 - (probA / (probA + probB))  else (teamB, teamA)
    else:
        return teamA if random.random() < 1 - (probA / (probA + probB))  else teamB

## Tournament Model

In [58]:
def tournament():
    ## Reset scores:
    for pool in [PoolA, PoolB, PoolC, PoolD]:
        for team in pool.keys():
            pool[team][2] = 0
    ## Set up the quarter/semi/final lists:
    quarter = []
    semi = []
    final = []
    bronze = []
    ## We play the pool games first:
    for pool in [PoolA, PoolB, PoolC, PoolD]:
        for game in combinations(pool.keys(), 2):
            win = who_won(game[0], game[1], pool[game[0]][0], pool[game[1]][0])
            pool[win][2] += 30
        order = [pool[k] for k in sorted(pool, key=lambda k: pool[k][1], reverse=True)]
        max_score = order[0][2]# Best score
        winner = random.choice([team for team in pool.keys() if pool[team][2] == max_score])   # Who won
        pool[winner][2] += 30  # Gets 30pts
        quarter.append(winner) # Add to quarter finals
        second_score = order[0][1]      
        second = random.choice([team for team in pool.keys() if pool[team][2] == second_score and team != winner])   # Who won              # Who came second
        pool[second][2] += 15  # Gets 15pts
        quarter.append(second) # Add to quarter finals
    ## Combine the pools into a full set:
    Pools = Tnorm({**PoolA, **PoolB, **PoolC, **PoolD})
    ## Next play the quarter finals:
    for qp in quarter_play:
        teamA = quarter[qp[0]]
        teamB = quarter[qp[1]]
        winner = who_won(teamA, teamB, Pools[teamA][1], Pools[teamB][1])
        Pools[winner][2] += 40
        semi.append(winner)
    ## Next play the semi finals:
    for sp in semi_play:
        teamA = semi[sp[0]]
        teamB = semi[sp[1]]
        winner, loser = who_won(teamA, teamB, Pools[teamA][1], Pools[teamB][1], True)
        Pools[winner][2] += 80
        final.append(winner)
        bronze.append(loser)
    ## Play for bronze:
    Pools[who_won(bronze[0], bronze[1], Pools[bronze[0]][1], Pools[bronze[1]][1])][2] += 40
    ## Play the final:
    Pools[who_won(final[0], final[1], Pools[final[0]][1], Pools[final[1]][1])][2] += 120
    
    return({team:Pools[team][2] for team in Teams})

In [59]:
def score(shares, total_shares, T):
    total = 0
    for team in shares.keys():
        total += T[team] * (shares[team] / total_shares[team])
    return total

And so, given we run a tournament and own the shares given by 'shares', if everyone else owns the shares given by 'total_shares', then we can work out our score. For example:

In [60]:
T = tournament()
shares = {'England':4, 'New Zealand':2, 'South Africa':2, 'Fiji':4, 'France':4}
total_shares = {'Ireland': 10, 'Scotland': 3, 'Japan': 20, 'Russia': 5, 'Samoa': 8, 'New Zealand': 30, 'South Africa': 50, 'Italy': 12, 'Namibia': 0, 'Canada': 3, 'England': 35, 'France': 30, 'Argentina': 30, 'Tonga': 4, 'USA': 5, 'Wales': 30, 'Australia': 3, 'Fiji': 3, 'Georgia': 3, 'Uruguay': 6}
score(shares, total_shares, T)

137.9142857142857

## Tactics
Define various tactics that people may employ:

In [61]:
## Go for full 4 shares of 4 "good" teams
def good_tactic():
    good_teams = ['New Zealand', 'England', 'Wales', 'South Africa', 'Ireland', 'Australia']
    return Counter({r:4 for r in random.sample(good_teams, 4)})

## Spread out shares on "good" teams
def spread_tactic():
    middle_teams = ['New Zealand', 'England', 'Wales', 'South Africa', 'Ireland', 'Australia']
    shares = {team:4 for team in middle_teams} # Start with all shares on 4
    while sum([i for i in shares.values()]) != 16:
        team = random.choice([i for i in shares.keys()])     # Choose a random team
        shares[team] -= 1               # Decrease the share by 1
        if shares[team] == 0:           # If it hits 0, delete it
            del shares[team]
    return Counter(shares)
    
## Return shares of random teams
def random_tactic():
    shares = {team:4 for team in Teams} # Start with all shares on 4
    while sum([i for i in shares.values()]) != 16:
        team = random.choice([i for i in shares.keys()])     # Choose a random team
        shares[team] -= 1               # Decrease the share by 1
        if shares[team] == 0:           # If it hits 0, delete it
            del shares[team]
    return Counter(shares)
    
## Return shares of "middle teams"
def middle_tactic():
    middle_teams = ['Australia', 'France', 'Scotland', 'Argentina', 'Japan', 'Fiji']
    shares = {team:4 for team in middle_teams} # Start with all shares on 4
    while sum([i for i in shares.values()]) != 16:
        team = random.choice([i for i in shares.keys()])     # Choose a random team
        shares[team] -= 1               # Decrease the share by 1
        if shares[team] == 0:           # If it hits 0, delete it
            del shares[team]
    return Counter(shares)
    
## Go for full 4 shares of 4 teams across pools
def pool_tactic():
    shares = {}
    for pool in [PoolA, PoolB, PoolC, PoolD]:
        team = random.choice([i for i in pool.keys()])
        shares[team] = 4
    return Counter(shares)

## Ben tactic: 4 on NZ and SA.
def ben_tactic():
    good_teams = ['New Zealand', 'England', 'Wales', 'South Africa', 'Ireland', 'Australia', 'France', 'Scotland', 'Argentina', 'Japan', 'Fiji']
    shares = {team:4 for team in good_teams} # Start with all shares on 4
    while sum([i for i in shares.values()]) != 16:
        team = random.choice([i for i in shares.keys()])     # Choose a random team
        if team == 'New Zealand' or team == 'South Africa':
            continue
        shares[team] -= 1               # Decrease the share by 1
        if shares[team] == 0:           # If it hits 0, delete it
            del shares[team]
    return Counter(shares)

In [62]:
def run_game(tactics, shares, n, m, spread = False):
    S = []
    for i in range(n):
        T = tournament()
        if not spread:
            for j in range(m):
                total_shares = Counter(shares)
                for t1 in range(tactics[0]):
                    total_shares += good_tactic()
                for t2 in range(tactics[1]):
                    total_shares += random_tactic()
                for t3 in range(tactics[2]):
                    total_shares += middle_tactic()
                for t4 in range(tactics[3]):
                    total_shares += pool_tactic()
                S.append(score(shares, total_shares, T))
        else:
            total_shares = Counter(shares)
            for t1 in range(tactics[0]):
                    total_shares += good_tactic()
            for t2 in range(tactics[1]):
                total_shares += random_tactic()
            for t3 in range(tactics[2]):
                total_shares += middle_tactic()
            for t4 in range(tactics[3]):
                total_shares += pool_tactic()
            for t5 in range(tactics[4]):
                total_shares += spread_tactic()
            S.append(score(shares, total_shares, T))
    return sum(S) / len(S)     

Lots of people go for good teams:

In [63]:
tactics = [10,5,3,2]

for i in range(100):
    shares = random_tactic()
    s = run_game(tactics, shares, 100, 10)
    if s > 100:
        print(shares)
        print(s)
        print('\n')

Counter({'Scotland': 2, 'Italy': 2, 'Namibia': 2, 'England': 2, 'France': 2, 'Wales': 2, 'Russia': 1, 'South Africa': 1, 'Canada': 1, 'USA': 1})
100.67540920380607


Counter({'Japan': 3, 'Scotland': 2, 'Namibia': 2, 'Australia': 2, 'Ireland': 1, 'Samoa': 1, 'New Zealand': 1, 'South Africa': 1, 'Italy': 1, 'Argentina': 1, 'USA': 1})
100.00608157600988


Counter({'Scotland': 3, 'France': 3, 'Japan': 2, 'New Zealand': 2, 'South Africa': 2, 'Ireland': 1, 'Italy': 1, 'Canada': 1, 'Argentina': 1})
114.6883665252083


Counter({'Ireland': 3, 'Scotland': 3, 'Argentina': 3, 'Wales': 2, 'Uruguay': 2, 'Samoa': 1, 'England': 1, 'Georgia': 1})
100.27240110193848


Counter({'New Zealand': 3, 'South Africa': 2, 'France': 2, 'Argentina': 2, 'Uruguay': 2, 'Scotland': 1, 'Samoa': 1, 'Wales': 1, 'Fiji': 1, 'Georgia': 1})
103.5106283795287




Lots of people go for middle teams:

In [477]:
tactics = [5,5,10,2]

for i in range(100):
    shares = random_tactic()
    s = run_game(tactics, shares, 100, 10)
    if s > 100:
        print(shares)
        print(s)
        print('\n')

Counter({'Japan': 3, 'England': 3, 'Scotland': 2, 'France': 2, 'Wales': 2, 'Samoa': 1, 'New Zealand': 1, 'South Africa': 1, 'Fiji': 1})
104.84689381728379


Counter({'Ireland': 2, 'Samoa': 2, 'Canada': 2, 'USA': 2, 'Georgia': 2, 'Russia': 1, 'New Zealand': 1, 'South Africa': 1, 'Italy': 1, 'England': 1, 'France': 1})
102.12293975967269


Counter({'England': 4, 'Scotland': 2, 'Japan': 2, 'USA': 2, 'Samoa': 1, 'New Zealand': 1, 'South Africa': 1, 'Italy': 1, 'Tonga': 1, 'Fiji': 1})
103.85987990698501


Counter({'New Zealand': 4, 'Ireland': 3, 'South Africa': 2, 'Argentina': 2, 'Wales': 2, 'France': 1, 'Tonga': 1, 'Georgia': 1})
133.75163102576423


Counter({'England': 4, 'Russia': 2, 'Samoa': 2, 'Wales': 2, 'Uruguay': 2, 'Japan': 1, 'New Zealand': 1, 'Italy': 1, 'Fiji': 1})
105.09529802612198


Counter({'New Zealand': 2, 'South Africa': 2, 'Italy': 2, 'Namibia': 2, 'England': 2, 'Georgia': 2, 'Ireland': 1, 'France': 1, 'USA': 1, 'Uruguay': 1})
120.44135127285045


Counter({'New Zealand':

Lots of people are random:

In [486]:
tactics = [4,10,4,2]

for i in range(100):
    shares = random_tactic()
    s = run_game(tactics, shares, 100, 10)
    if s > 100:
        print(shares)
        print(s)
        print('\n')

Counter({'England': 3, 'France': 3, 'USA': 3, 'Wales': 3, 'New Zealand': 2, 'Ireland': 1, 'Fiji': 1})
112.98756274877083


Counter({'Scotland': 2, 'New Zealand': 2, 'Italy': 2, 'Ireland': 1, 'Japan': 1, 'Samoa': 1, 'South Africa': 1, 'Namibia': 1, 'Canada': 1, 'England': 1, 'Argentina': 1, 'Wales': 1, 'Uruguay': 1})
104.61683066876947


Counter({'Australia': 3, 'Japan': 2, 'New Zealand': 2, 'Italy': 2, 'Ireland': 1, 'Scotland': 1, 'Samoa': 1, 'South Africa': 1, 'Argentina': 1, 'Wales': 1, 'Fiji': 1})
102.25292848141666


Counter({'England': 3, 'Ireland': 2, 'South Africa': 2, 'Italy': 2, 'France': 2, 'Georgia': 2, 'Canada': 1, 'USA': 1, 'Wales': 1})
102.42661741409607


Counter({'England': 3, 'Ireland': 2, 'Scotland': 2, 'Samoa': 2, 'Italy': 2, 'South Africa': 1, 'Canada': 1, 'France': 1, 'USA': 1, 'Wales': 1})
102.41889939756771


Counter({'Ireland': 3, 'New Zealand': 2, 'Namibia': 2, 'Wales': 2, 'Japan': 1, 'South Africa': 1, 'England': 1, 'Argentina': 1, 'Tonga': 1, 'USA': 1, 'Austr

Lots of people spread out:

In [487]:
tactics = [6,4,5,8]

for i in range(100):
    shares = random_tactic()
    s = run_game(tactics, shares, 100, 10)
    if s > 100:
        print(shares)
        print(s)
        print('\n')

Counter({'England': 3, 'Wales': 3, 'South Africa': 2, 'Australia': 2, 'Ireland': 1, 'New Zealand': 1, 'Namibia': 1, 'France': 1, 'Argentina': 1, 'Tonga': 1})
102.12412703682732


Counter({'New Zealand': 2, 'South Africa': 2, 'Italy': 2, 'England': 2, 'France': 2, 'Wales': 2, 'Samoa': 1, 'Canada': 1, 'USA': 1, 'Fiji': 1})
100.1731307426265




In [531]:
o = {'New Zealand': 4, 'South Africa': 4, 'Ireland': 3, 'England': 2, 'Wales': 2, 'Scotland': 1}
for tactics in all_tactics:  
        print(run_game(tactics, o, 100, 10))

105.59377850550342
162.9255571296172
153.73725119493625
125.4826640244673


In [534]:
o = {'New Zealand': 4, 'England': 4, 'South Africa': 4, 'Wales': 2, 'Ireland': 2}
for tactics in all_tactics:  
        print(run_game(tactics, o, 100, 10))

102.33613062293999
167.46181719423103
157.65107560999277
127.90140792451375


In [533]:
o = {'New Zealand': 4, 'South Africa': 4, 'England': 3, 'Wales': 2, 'Australia': 2, 'Ireland': 1}
for tactics in all_tactics:  
        print(run_game(tactics, o, 100, 10))

100.89123938221138
159.25852667342534
153.84922926535006
125.16024681760454


In [538]:
o = {'New Zealand': 4, 'England': 4, 'South Africa': 4, 'Wales': 2, 'France': 1, 'Scotland': 1}
for tactics in all_tactics:  
        print(run_game(tactics, o, 100, 10))

107.57119230429818
153.6887583592056
153.6486384407959
123.86109203183526


In [539]:
o = {'New Zealand': 4, 'South Africa': 4, 'Wales': 3, 'England': 2, 'Ireland': 2, 'Australia': 1}
for tactics in all_tactics:  
        print(run_game(tactics, o, 100, 10))

100.21020575583027
166.51975030782714
156.59525540221344
127.05265746574884


## Final tests with spread

In [566]:
o1 = {'Ireland': 4, 'New Zealand': 3, 'Scotland': 2, 'Italy': 2, 'Wales': 2, 'Canada': 1, 'England': 1, 'France': 1}
o2 = {'New Zealand': 4, 'Ireland': 3, 'South Africa': 2, 'Argentina': 2, 'Wales': 2, 'France': 1, 'Tonga': 1, 'Georgia': 1}
o3 = {'New Zealand': 4, 'South Africa': 4, 'England': 3, 'Wales': 3, 'Ireland': 1, 'Scotland': 1}
o4 = {'New Zealand': 4, 'England': 4, 'South Africa': 4, 'Wales': 2, 'France': 1, 'Scotland': 1}
o5 = {'New Zealand': 4, 'South Africa': 4, 'Wales': 3, 'England': 2, 'Ireland': 2, 'Australia': 1}
o6 = {'New Zealand': 4, 'England': 4, 'South Africa': 4, 'Wales': 2, 'Ireland': 2}

options = [o1,o2,o3,o4,o5,o6]

all_tactics = [[3,5,5,2,3]] # Change this here based on office tactic proportions
scores = [0,0,0,0,0,0]

for i,shares in enumerate(options):
    for tactics in all_tactics:  
        scores[i] += run_game(tactics, shares, 100, 10, True)
        
print(scores)


[126.64514289629, 127.2811865169188, 143.99178058790466, 142.1425235870092, 145.8307830597511, 146.2142155259393]


## ELO

In [64]:
## Probability that TeamA wins:
def Exp(RankA, RankB):
    return 1 / (1 + 10**((RankB - RankA)/400))

In [528]:
wins = {'England':0, 'France':0, 'Argentina':0, 'Tonga':0, 'USA':0}
for i in range(10**6):
    pool = {'England':[2245,0], 'France':[2110,0], 'Argentina':[2100,0], 'Tonga':[1810,0], 'USA':[1810,0]}
    for game in combinations(pool.keys(), 2):
        win = game[0] if random.random() < Exp(pool[game[0]][0], pool[game[1]][0]) else game[1]
        pool[win][1] += 30
    order = [pool[k] for k in sorted(pool, key=lambda k: pool[k][1], reverse=True)]
    max_score = order[0][1]# Best score
    wins[random.choice([team for team in pool.keys() if pool[team][1] == max_score])] += 1   # Who won

In [529]:
wins

{'England': 554772,
 'France': 221603,
 'Argentina': 205534,
 'Tonga': 9103,
 'USA': 8988}

In [468]:
odds = [10**6 * s[0] for s in PoolC.values()]

In [469]:
for i in range(5):
    print(0.001*(odds[i] - list(wins.values())[i]))

-17.227649148038356
3.9153900313762424
13.8925255799593
-0.26513323164858615
-0.31513323164858614


In [457]:
odds

[692705.3508519616,
 170702.39003137624,
 132768.5255799593,
 1911.8667683514138,
 1911.8667683514138]